# Arithmetic and Algorithms: From Z/nZ to Elliptic Curves
**Author :** AESpider  

This notebook explores the mathematical foundations of public-key cryptography. It is divided into four parts:

1.  **TD01 :** Complexity, numerical computation, and graphs.
2.  **TD02 :** Finite fields, primality, and discrete logarithm.
3.  **TD03 :** Factorization and RSA cryptanalysis.
4.  **TD04 :** Elliptic Curves and advanced attacks (Smart's, Lattice, ElGamal DFA).

## TD01: Complexity, Numerical Computation, and Graphs

### 1.Time complexity analysis
Comparative measurement of elementary arithmetic and matrix operations.

In [1]:
import time

def measure_time(func, *args):
    start = time.time()
    func(*args)
    return time.time() - start

print("Addition vs multiplication integers")
for bits in [10000, 50000, 100000, 200000]:
    a = ZZ.random_element(2^bits)
    b = ZZ.random_element(2^bits)
    
    t_add = measure_time(lambda: a + b)
    t_mul = measure_time(lambda: a * b)
    
    print(f"Bits: {bits} | Add: {t_add:.6f}s | Mul: {t_mul:.6f}s")

print("\nModular exponentiation")
for bits in [1000, 2000, 4000]:
    a = ZZ.random_element(2^bits) 
    b = ZZ.random_element(2^bits)
    c = ZZ.random_element(2^bits)
    
    t_pow = measure_time(pow, a, b, c) # (a^b mod c)
    print(f"Bits: {bits} | Time: {t_pow:.6f}s")

print("\nMatrix multiplication in Z/nZ")
mod_n = 100
R = Integers(mod_n)
for dim in [50, 100, 200]:
    MS = MatrixSpace(R, dim)
    M1 = MS.random_element()
    M2 = MS.random_element()
    
    t_mat = measure_time(lambda: M1 * M2)
    print(f"Dim: {dim}x{dim} | Time: {t_mat:.6f}s")

Addition vs multiplication integers
Bits: 10000 | Add: 0.000005s | Mul: 0.000027s
Bits: 50000 | Add: 0.000005s | Mul: 0.000221s
Bits: 100000 | Add: 0.000008s | Mul: 0.000822s
Bits: 200000 | Add: 0.000006s | Mul: 0.010736s

Modular exponentiation
Bits: 1000 | Time: 0.007950s
Bits: 2000 | Time: 0.007617s
Bits: 4000 | Time: 0.035678s

Matrix multiplication in Z/nZ
Dim: 50x50 | Time: 0.003880s
Dim: 100x100 | Time: 0.003489s
Dim: 200x200 | Time: 0.011722s


### 2. Numerical approximation of gaussian integral

In [2]:
f(x) = exp(-x^2)

# compute numerical integral from -infinity to +infinity
result, error = numerical_integral(f, -infinity, infinity)

print(f"Numerical result: {result:.10f}")
print(f"Exact sqrt(pi):   {sqrt(pi).n():.10f}")
print(f"Difference:       {abs(result - sqrt(pi).n()):.10e}")

Numerical result: 1.7724538509
Exact sqrt(pi):   1.7724538509
Difference:       2.2204460493e-15


### 3. Sorting algorithms
Performance comparison between bubble sort ($O(n^2)$) and merge sort ($O(n \log n)$).

In [3]:
# Slow
def bubble_sort(arr):
    n = len(arr)
    for i in range(n):
        for j in range(0, n-i-1):
            if arr[j] > arr[j+1]:
                arr[j], arr[j+1] = arr[j+1], arr[j]
    return arr

# Fast
def merge_sort(arr):
    if len(arr) <= 1:
        return arr
    mid = len(arr) // 2
    left = merge_sort(arr[:mid])
    right = merge_sort(arr[mid:])
    
    return merge(left, right)

def merge(left, right):
    result = []
    i = j = 0
    while i < len(left) and j < len(right):
        if left[i] < right[j]:
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1
    result.extend(left[i:])
    result.extend(right[j:])
    return result

list_size = 2000
random_list = [randint(0, 10000) for _ in range(list_size)]
list_copy = list(random_list)

t_slow = measure_time(bubble_sort, random_list)
t_fast = measure_time(merge_sort, list_copy)

print(f"List size: {list_size}")
print(f"Bubble Sort : {t_slow:.6f}s")
print(f"Merge Sort  :  {t_fast:.6f}s")

List size: 2000
Bubble Sort : 4.724890s
Merge Sort  :  0.028653s


### 4. Graphs: connected component
Breadth-First Search (BFS) algorithm to identify disjoint subgraphs.

In [4]:
def get_connected_component(graph, start_vertex):
    visited = set()
    queue = [start_vertex]
    visited.add(start_vertex)
    component = []
    
    while queue:
        vertex = queue.pop(0)
        component.append(vertex)
        
        for neighbor in graph.neighbors(vertex):
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(neighbor)
                
    return component

# create a graph with 2 disjoint components
G = Graph({0:[1, 2], 1:[2], 3:[4]}) 
start_node = 0
comp = get_connected_component(G, start_node)

print(f"Graph edges: {G.edges(labels=False)}")
print(f"Connected component containing {start_node}: {comp}")

Graph edges: [(0, 1), (0, 2), (1, 2), (3, 4)]
Connected component containing 0: [0, 1, 2]


## TD02 : Finite fields, primality, and discrete logarithm

### 1. Modular arithmetic and Carmichael numbers
Fast modular exponentiation and discovery of pseudoprimes.

In [5]:
res = power_mod(2, 12345678987654321, 101)
print(f"2^12345678987654321 mod 101 = {res}")

N = 561
print(f"is {N} composite? {not is_prime(N)}")

is_carmichael = True
for x in range(1, N):
    if gcd(x, N) == 1:
        if power_mod(x, N - 1, N) != 1:
            is_carmichael = False
            break
            
print(f"is {N} a Carmichael number? {is_carmichael}")

2^12345678987654321 mod 101 = 89
is 561 composite? True
is 561 a Carmichael number? True


### 2. Z/NZ and Z/PZ properties
Study of invertible elements, cyclicity, and logarithm tables.

In [6]:
R = Integers(35)
invertibles = [x for x in R if x.is_unit()]
print(f"Invertibles in Z/35Z: {invertibles}")

print(f"is (Z/35Z)* cyclic? {R.multiplicative_group_is_cyclic()}")

# find element of maximal order
orders = [(x, x.multiplicative_order()) for x in invertibles]
max_order_elt = max(orders, key=lambda item: item[1])
print(f"Max order element: {max_order_elt[0]} (Order: {max_order_elt[1]})")

# Z/11Z generator and tables
p = 11
F = GF(p)
g = F.multiplicative_generator()
print(f"\nGenerator of Z/{p}Z: {g}")

print("Exponential table:")
for i in range(1, p):
    print(f"{g}^{i} = {g^i}")
    
print("\nLogarithm table:")
for x in range(1, p):
    print(f"log_{g}({x}) = {F(x).log(g)}")

Invertibles in Z/35Z: [1, 2, 3, 4, 6, 8, 9, 11, 12, 13, 16, 17, 18, 19, 22, 23, 24, 26, 27, 29, 31, 32, 33, 34]
is (Z/35Z)* cyclic? False
Max order element: 2 (Order: 12)

Generator of Z/11Z: 2
Exponential table:
2^1 = 2
2^2 = 4
2^3 = 8
2^4 = 5
2^5 = 10
2^6 = 9
2^7 = 7
2^8 = 3
2^9 = 6
2^10 = 1

Logarithm table:
log_2(1) = 0
log_2(2) = 1
log_2(3) = 8
log_2(4) = 2
log_2(5) = 4
log_2(6) = 9
log_2(7) = 7
log_2(8) = 3
log_2(9) = 6
log_2(10) = 5


### 3. Large prime and generator of (Z/NZ)*
Search for large prime numbers $p$ and generators for the group $(\mathbb{Z}/p\mathbb{Z})^*$.

In [7]:
# find p in [10^100, 2*10^100] such that (p-1)/2 is prime
low = 10**100
high = 2 * 10**100

while True:
    # generate q such that p = 2q + 1 falls in range
    q = random_prime(high // 2, lbound=low // 2)
    p = 2 * q + 1
    if is_prime(p):
        break
        
print(f"Found prime p: {p}")
print(f"(p-1)/2 is prime: {is_prime((p-1)//2)}")

# find generator for Z/pZ (order is 2q, factors are 2 and q)
g = 2
while True: 
    # check if order is p-1
    if power_mod(g, 2, p) != 1 and power_mod(g, q, p) != 1:
        break
    g += 1
    
print(f"Generator: {g}")

Found prime p: 15470771863470608431125264570486864316935591297785517163370885665883592659803713512838511099992471647
(p-1)/2 is prime: True
Generator: 5


In [8]:
def find_generator(N):
    if not is_prime(N):
        raise ValueError("N must be prime")
    
    factors_of_order = prime_factors(N - 1)
    
    while True:
        # pick a random element in [2, N-1]
        g = randint(2, N - 1)
        
        is_generator = True
        # check if g^((N-1)/q) == 1 mod N for any factor q
        for q in factors_of_order:
            if power_mod(g, (N - 1) // q, N) == 1:
                is_generator = False
                break
        
        if is_generator:
            return g

prime_N = 10007
gen = find_generator(prime_N)
print(f"Prime N: {prime_N}")
print(f"Generator found: {gen}")
print(f"Verification ({gen}^({prime_N-1}) mod N) = {power_mod(gen, prime_N-1, prime_N)}")

Prime N: 10007
Generator found: 8117
Verification (8117^(10006) mod N) = 1


### 4. Miller-Rabin primality test


In [9]:
def miller_rabin(n, k=10):
    if n == 2 or n == 3: return True
    if n % 2 == 0 or n < 2: return False
    
    # write n-1 as 2^r * d
    r, d = 0, n - 1
    while d % 2 == 0:
        r += 1
        d //= 2
        
    for _ in range(k):
        a = randint(2, n - 2)
        x = power_mod(a, d, n)
        
        if x == 1 or x == n - 1:
            continue
        
        composite = True
        for _ in range(r - 1):
            x = power_mod(x, 2, n)
            if x == n - 1:
                composite = False
                break
        
        if composite:
            return False
            
    return True # probably prime# TD03

print(f"Miller-Rabin(561): {miller_rabin(561)}")
print(f"Miller-Rabin(10007): {miller_rabin(10007)}")
print(f"Miller-Rabin(Next Prime): {miller_rabin(next_prime(10007))}")

Miller-Rabin(561): False
Miller-Rabin(10007): True
Miller-Rabin(Next Prime): True


### 5. Pohlig-Hellman
Reduction of the Discrete Logarithm Problem (DLP) when the group order is smooth.

In [10]:
def baby_step_giant_step(g, h, p, order):
    m = Integer(order).isqrt() + 1
    
    # Baby steps: store g^j
    table = {power_mod(g, j, p): j for j in range(m)}
    
    # Giant steps: check h * (g^-m)^i
    factor = power_mod(g, -m, p)
    candidate = h
    
    for i in range(m):
        if candidate in table:
            return i * m + table[candidate]
        candidate = (candidate * factor) % p
    return None

def chinese_remainder_theorem(remainders, moduli):
    M = prod(moduli)
    result = 0
    for ai, ni in zip(remainders, moduli):
        Mi = M // ni
        # compute inverse
        _, yi, _ = xgcd(Mi, ni)
        result += ai * yi * Mi
    return result % M

def pohlig_hellman(g, h, p):
    # use Sage to factor group order
    factors = list((p - 1).factor())
    remainders = []
    moduli = []
    
    print(f"Factors of p-1: {factors}\n")    
    for q, e in factors:
        qe = q^e
        # project to subgroup
        exponent = (p - 1) // qe
        gi = power_mod(g, exponent, p)
        hi = power_mod(h, exponent, p)
        
        # solve DLP in subgroup using our BSGS
        xi = baby_step_giant_step(gi, hi, p, qe)
        
        remainders.append(xi)
        moduli.append(qe)
        print(f"Log mod {qe}: {xi}")
        
    # reconstruct using our CRT
    return chinese_remainder_theorem(remainders, moduli)

In [11]:
# p-1 is smooth (its prime factors are small)
p = 413559242670505047035805476762189028817269047242988661147500669
g = 2
y = 3159994416106411411027414670767385809029780616457650050055396

# p = 94487163297382863167633060898467898176295383252944606346612565696030376662261885354113923956999751538280705847409180073365038322172781858474622022808482972630746560742341702742178330138682963971041358923639916031988458406627047092978738794000171785975693222202416488063109972907266960217738849683470100180657
# g = 91837172179571476776190849460700915868439517079156440218651607148606324494298737979207246124168621805457673911794964841647324903434381292641072254574996422082928102274796072341535153901982527988318666060710699023247733315763656131213319397976417698616067127741385175318280373623657263470144195815929916049604
# y = 80905650118846385931764925970457972889929953376763445484237320000327884329168934728928081077667420611873474478780456122831059312737019466367282871744034214492809575109792401639770538955432531764305943142688619101553963448240387184302532415259921365582743379207582614300826999903808215005060226437928891565763

print("Solving DLP with Pohlig-Hellman...")
x = pohlig_hellman(g, y, p)

print(f"\nRecovered x: {x}")
print(f"Verify ? {power_mod(g, x, p) == y}")

Solving DLP with Pohlig-Hellman...
Factors of p-1: [(2, 2), (29, 3), (43, 2), (59, 3), (101, 1), (109, 3), (139, 2), (263, 2), (283, 1), (313, 3), (317, 1), (457, 1), (509, 3), (683, 1), (751, 2)]

Log mod 4: 0
Log mod 24389: 3520
Log mod 1849: 1485
Log mod 205379: 39474
Log mod 101: 57
Log mod 1295029: 1271748
Log mod 19321: 15883
Log mod 69169: 26706
Log mod 283: 229
Log mod 30664297: 1271748
Log mod 317: 261
Log mod 457: 374
Log mod 131872229: 1271748
Log mod 683: 2
Log mod 564001: 143746

Recovered x: 1271748
Verify ? True


## TD03 : Factorization and RSA cryptanalysis.

### 1. Pollard's Rho 
Probabilistic factorization method based on cycle detection.

In [12]:
def pollard_rho(n):
    # f(x) = x^2 + 1 mod n
    x = 2; y = 2; d = 1
    
    while d == 1:
        # move x once: x = f(x)
        x = (x^2 + 1) % n
        
        # move y twice: y = f(f(y))
        y = (y^2 + 1) % n
        y = (y^2 + 1) % n
        
        d = gcd(abs(x - y), n)
        
    if d == n:
        # failed to find a non-trivial factor
        return None
    
    return d

p = 104729   
q = 1299721
N = p * q

print(f"Factoring N = {N}")
start_time = walltime()
factor = pollard_rho(N)
end_time = walltime()

print(f"Found factor: {factor}")
print(f"Other factor: {N // factor}")
print(f"Time taken:   {end_time - start_time:.6f}s")

Factoring N = 136118480609
Found factor: 104729
Other factor: 1299721
Time taken:   0.001133s


### 2. Security analysis (key length)
Estimation of theoretical complexity to break RSA based on the modulus size.

In [13]:
# complexity is O(N^(1/4)). 
# we estimate operations for standard RSA key sizes.

print("Estimated operations (N^(1/4)) for security break:")
for bits in [512, 1024, 2048]:
    # 2^bits is N, so operations is (2^bits)^(1/4) = 2^(bits/4)
    log_ops = bits / 4
    print(f"RSA-{bits} ~ 2^{int(log_ops)} operations")

Estimated operations (N^(1/4)) for security break:
RSA-512 ~ 2^128 operations
RSA-1024 ~ 2^256 operations
RSA-2048 ~ 2^512 operations


### 3. RSA Wiener's attack
Recovering the private key $d$ when it is too small ($d < \frac{1}{3}N^{1/4}$) via continued fractions.

In [14]:
def wiener_attack(e, N):
    """
    Recover d using continued fractions of e/N.
    Approximation: e/N ~ k/d
    """
    cf = continued_fraction(e/N)
    convergents = cf.convergents()
    
    for c in convergents:
        k = c.numerator()
        d = c.denominator()
        
        if k == 0: continue
        
        # potential phi = (ed - 1) / k
        # check divisibility
        if (e*d - 1) % k != 0:
            continue
            
        phi_cand = (e*d - 1) // k
        
        # check quadratic equation roots: x^2 - (N - phi + 1)x + N = 0
        # if roots are integer (p and q), then d is correct.
        b = N - phi_cand + 1
        delta = b^2 - 4*N
        
        if delta >= 0 and delta.is_square():
            return d
            
    return None

In [15]:
# setup RSA with a vulnerable key (small d)
# Wiener's attack works if d < (1/3) * N^(1/4)
n_bits = 1024
p = random_prime(2^(n_bits//2))
q = random_prime(2^(n_bits//2))
N = p * q
phi = (p-1)*(q-1)

# we force d to be small (approx N^0.24)
d_target = random_prime(integer_floor(N^0.24))
e = inverse_mod(d_target, phi)

print(f"Modulus N: {N}")
print(f"Public e:  {e}")
print(f"Target d:  {d_target}")

Modulus N: 12020674271020522985910166886589228501532637370003219926915392042032595736856152653323448466317299709071562305881209106874676458412662744635574440342743371809522587294218906986225708162350117204786725741033307658512789247448057471618380081841310221179759998173450325980516963429437863381622682135375543484447
Public e:  6488147680521248607099481314943544944398757089591900345930038439088329124428202015035107823109547243114576979702701388677484599635852943246053267193104371402417909948779842435797402839908107977796677065211755086886481975956952154771671466201669680363856804859945039495907024706496084113317471861337004204419
Target d:  28360418554732160124000455763910851866550231612940896371451089329213418123


In [16]:
print("Running Wiener's Attack...")
d_found = wiener_attack(e, N)

print(f"Recovered d: {d_found}")
print(f"Match ? {d_found == d_target}")

Running Wiener's Attack...
Recovered d: 28360418554732160124000455763910851866550231612940896371451089329213418123
Match ? True


### Bonus: RSA key recovery (Coppersmith's small roots)
Using lattice reduction to recover a factor $p$ when a portion of its bits is known.

In [17]:
# Problem: we know the high bits of a prime factor p.
# we recover the full p using Coppersmith's method (Lattice reduction).

# create a vulnerable RSA key
# we use a 1024-bit modulus. p and q are 512 bits.
n_bits = 1024
p = random_prime(2^(n_bits//2-1), lbound=2^(n_bits//2-2))
q = random_prime(2^(n_bits//2-1), lbound=2^(n_bits//2-2))
N = p * q

# we simulate a "leak": we only know the upper 60% of bits of p
mask_bits = int(512 * 0.4) # we hide lower 40%
mask = (1 << mask_bits) - 1
p_known = p - (p & mask)

print(f"Modulus N: {N}")
print(f"p (Target): {p}")
print(f"p_known:    {p_known} (Lower {mask_bits} bits hidden)")

Modulus N: 19790070231323186944630199721807107279305057338908252115184023335524151512787975621597494546413750833935094111179803003682549239625011125059446662450568515091735584393429851618023269615030295895242441137897128586097262332273073021925139213239469383593239518595145713300948988602632619502414275467808253362113
p (Target): 3633124237110238197231968488071998045505086105452161591806708102269836138091809285538478472827369606096814835102509986198591950144204767915559736733268567
p_known:    3633124237110238197231968488071998045505086105452161591806708102269836138091809285538478472821396397584165319755873315009371230889548368774910770776375296 (Lower 204 bits hidden)


In [18]:
# we look for x such that f(x) = p_known + x = 0 mod p and |x| < 2^mask_bits
R.<x> = PolynomialRing(Zmod(N))
f = p_known + x

# builtin small_roots which implements Coppersmith/LLL
# beta=0.5 means we look for a factor >= N^0.5 (which p is)
print("Running Coppersmith's small_roots...\n")
roots = f.small_roots(X=2^mask_bits, beta=0.4)

if roots:
    p_recovered = p_known + int(roots[0])
    print(f"Recovered p: {p_recovered}")
    print(f"Match ? {p_recovered == p}")
else:
    print("Failed to find small root.")

Running Coppersmith's small_roots...

Recovered p: 3633124237110238197231968488071998045505086105452161591806708102269836138091809285538478472827369606096814835102509986198591950144204767915559736733268567
Match ? True


## TD04 : Elliptic Curves and advanced attacks (Smart's, Lattice, ElGamal DFA).

### 1. Elliptic curve over $\mathbb{F}_p$
Curve definition, point addition, and group structure.

In [19]:
# define the curve y^2 = x^3 + 2x + 1 over F7
F = GF(7)
E = EllipticCurve(F, [0, 0, 0, 2, 1])

print(f"is smooth? {E.is_smooth()}")
print(f"Points in E(F7): {E.points()}")

P = E(0, 6)
Q = E(1, 5)

print(f"2P = {2*P}")
print(f"P + Q = {P + Q}")

G = E.abelian_group()
print(f"Group structure: {G}")

is smooth? True
Points in E(F7): [(0 : 1 : 0), (0 : 1 : 1), (0 : 6 : 1), (1 : 2 : 1), (1 : 5 : 1)]
2P = (1 : 2 : 1)
P + Q = (0 : 1 : 1)
Group structure: Additive abelian group isomorphic to Z/5 embedded in Abelian group of points on Elliptic Curve defined by y^2 = x^3 + 2*x + 1 over Finite Field of size 7


### 2. Curves over extension fields $\mathbb{F}_{p^k}$

In [20]:
# define field K = F5[x]/(x^2+x+1)
K.<a> = GF(5^2, modulus=x^2+x+1)
print(f"Field K size: {K.order()}")

# curve y^2 = x^3 + x + 1 over K
D = EllipticCurve(K, [0, 0, 0, 1, 1])
print(f"is D smooth? {D.is_smooth()}")

P = D(4, 3)
Q = D(3*a + 1, 4*a + 2)

print(f"2P = {2*P}")
print(f"P + Q = {P + Q}")

Field K size: 25
is D smooth? True
2P = (3 : 1 : 1)
P + Q = (2*a + 2 : 4*a + 1 : 1)


### 3. Elliptic Curve Method Factorization
Using Lenstra's method to factor integers.

In [21]:
def modular_inverse(val, n):
    val_int = Integer(val)
    n_int = Integer(n)
    g, u, v = xgcd(val_int, n_int)
    
    if g > 1:
        return None, g # factor found
    return u % n, None

def elliptic_add(P1, P2, a, n):
    if P1 is None: return P2, None
    if P2 is None: return P1, None
    
    x1, y1 = P1
    x2, y2 = P2
    
    # check if doubling or addition
    if x1 == x2:
        if (y1 + y2) % n == 0:
            return None, None # point at infinity
        num = 3*x1^2 + a
        den = 2*y1
    else:
        num = y2 - y1
        den = x2 - x1
        
    inv, factor = modular_inverse(den, n)
    if factor:
        return None, factor
        
    lam = (num * inv) % n
    x3 = (lam^2 - x1 - x2) % n
    y3 = (lam*(x1 - x3) - y1) % n
    
    return (x3, y3), None

def ecm(n, B=50):
    if is_prime(n): return None
    R = Integers(n)
    
    # try random curves
    for _ in range(50):
        a = R.random_element()
        x = R.random_element()
        y = R.random_element()
        P = (x, y)
        
        # compute k! * P
        for k in range(2, B + 1):
            Q = P
            # repeated addition for scalar multiplication
            for _ in range(k - 1):
                Q, factor = elliptic_add(Q, P, a, n)
                if factor:
                    if 1 < factor < n: return factor
                    break
                if Q is None: break
            
            P = Q
            if P is None: break
            
    return None

n = 4223
factor = ecm(n, B=20)

print(f"Factoring n={n}")
print(f"Factor found: {factor}")
if factor:
    print(f"Other factor: {n//factor}")

Factoring n=4223
Factor found: 41
Other factor: 103


In [22]:
from sage.libs.libecm import ecmfactor

def ecm(n, B=20):
    if is_prime(n): return None
    
    # try different curves to avoid trivial factor n
    for sigma_val in range(10):
        res = ecmfactor(n, B, sigma=sigma_val)
        if res[0]: # second element is the factor
            factor = res[1]
            if 1 < factor < n:
                return factor
    return None

n = 4223
factor = ecm(n, 15) # reduced B for small n
print(f"Factoring n={n}")
print(f"Factor found: {factor}")
print(f"Other factor: {n//factor}")

Factoring n=4223
Factor found: 41
Other factor: 103


### 4. Weak Curves: Smart's attack
Solving DLP in linear time on anomalous curves (Trace of Frobenius = 1) via p-adic lifting.

In [23]:
"""
Smart's Attack on Anomalous Elliptic Curves
Paper: Weak Curves In Elliptic Curve Cryptography
https://wstein.org/edu/2010/414/projects/novotney.pdf
"""

def lift_point(P, p, prec):
    """
    Lift a point from F_p to Q_p using Hensel's lemma.
    
    Args:
        P: Point on elliptic curve over F_p
        p: Prime modulus
        prec: p-adic precision
    
    Returns:
        Lifted point in Q_p
    """
    E = P.curve()
    Eq = E.change_ring(QQ)
    Eqp = Eq.change_ring(Qp(p, prec))
    
    x_P, y_P = map(ZZ, P.xy())
    
    # build Weierstrass equation: g(y) = y^2 + a1*x*y + a3*y - x^3 - a2*x^2 - a4*x - a6
    y = var('y')
    g = (y**2 
         + Eq.a1()*x_P*y + Eq.a3()*y 
         - x_P**3 - Eq.a2()*x_P**2 - Eq.a4()*x_P - Eq.a6())
    g_prime = diff(g, y)
    
    # Hensel lifting via Newton iterations
    y_lift = y_P
    for i in range(1, prec):
        g_val = Integer(g.subs(y=y_lift))
        g_prime_val = Integer(g_prime.subs(y=y_lift))
        
        if gcd(g_prime_val, p) != 1:
            raise ValueError(f"Hensel lift failed: derivative not invertible at step {i}")
        
        # Newton: y := y - g(y)/g'(y) mod p^(i+1)
        y_lift = ZZ(Mod(y_lift - inverse_mod(g_prime_val, p**i) * g_val, p**(i+1)))
    
    return Eqp(Eqp.base_ring()(x_P), Eqp.base_ring()(y_lift))


def smart_attack(P, Q, p, prec=8):
    """
    Solve ECDLP on anomalous curves using Smart's attack.
    
    The attack exploits the p-adic elliptic logarithm on curves with trace of 
    Frobenius = 1 (i.e., #E(F_p) = p).
    
    Args:
        P: Base point
        Q: Target point (find k where Q = k*P)
        p: Prime (must equal curve order)
        prec: p-adic precision (default: 8)
    
    Returns:
        Discrete logarithm k such that Q = k*P
    """
    # lift points to p-adic field
    P_lifted = lift_point(P, p, prec)
    Q_lifted = lift_point(Q, p, prec)
    
    # compute p-adic elliptic logarithm: phi(P) = -x(pP) / y(pP)
    x_pP, y_pP = (p * P_lifted).xy()
    x_pQ, y_pQ = (p * Q_lifted).xy()
    
    phi_P = -x_pP / y_pP
    phi_Q = -x_pQ / y_pQ
    
    # discrete log is the ratio k = phi(Q) / phi(P)
    return Integer(Mod(phi_Q / phi_P, p))

In [24]:
# curve parameters
p = 37596521231081692286097719226317294959438237380966990611426250022860049363722901792938760858989180414389198817978737
a = 9980768944941555246412259964901404279221262365843204377450275584172813162519191945154105581021730623506736610561436
b = 6204302295427697621568533586763401593326903936424761465051089911662954021106621122465925081755300614989243091933253
    
# points
Gx = 15447065612664874603863922074709843217617947300821194509895141112376379689800058863402477048383987615075331617146523
Ax = 10577607971208571434119391805999854179572624626504025910910768124944865520669643879346931306961545644496867645040277
    
# setup
E = EllipticCurve(GF(p), [a, b])
G = E.lift_x(GF(p)(Gx))
A = E.lift_x(GF(p)(Ax))
    
if E.order() != p:
    print(f"Curve is NOT anomalous - attack not applicable")
    print(f"   Order: {E.order()}")
    print(f"   Prime: {p}")
    print(f"Smart's attack only works when #E(F_p) = p")
else: 
    print(f"Curve is anomalous (#E = p)")
    print(f"Solving A = d*G...\n")
    try:
        d = smart_attack(G, A, p, prec=8)
        if A == d * G: print(f"d = {d}")
        else: print(f"Verification failed: A != d*G")

    except Exception as e:
        print(f"Attack failed: {e}")

Curve is anomalous (#E = p)
Solving A = d*G...

d = 691373327119659988940370256685427003946313173177688134079438534648334833732624030914980677542622539902536846706034


### 5. ECDSA Hidden Number Problem (Lattice attack)
Solving the "Hidden Number Problem" to recover a private key from biased signatures (truncated nonce).

In [25]:
# Problem: ECDSA Nonce Bias.
# if the nonce k is small (e.g. 8 MSB are 0), we can recover d via HNP.

# setup curve and biased signatures
p = 0xfffffffffffffffffffffffffffffffffffffffffffffffffffffffefffffc2f
order = 0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141
E = EllipticCurve(GF(p), [0, 7])
G = E.gen(0)
d_target = randint(1, order-1) # private key
Q = d_target * G               # public key
print(f"Target Private Key: {d_target}")

# collect signature
signatures = []
num_sigs = 42
bits_known = 8 # we simulate small k
bound = order >> bits_known

for _ in range(num_sigs):
    k = randint(1, bound) # BIASED k
    R_point = k * G
    r = Integer(R_point[0])
    if r == 0: continue
    e_hash = randint(1, order-1)
    # ECDSA: s = k^-1 * (e + d*r)  =>  k = s^-1 * (e + d*r)
    s = inverse_mod(k, order) * (e_hash + d_target * r) % order
    signatures.append((r, s, e_hash))

# Construct Lattice for HNP
# equation: k - d * (r/s) - (e/s) = 0 mod n
# We want to find small k (and thus d).
# Matrix B format (Boneh-Venkatesan):
# [ n   0   0 ... 0 ]
# [ 0   n   0 ... 0 ]
# ...
# [ t1  t2 ... tn 2^bits/n ]  <-- t_i = r_i * s_i^-1 mod n
# [ u1  u2 ... un 0        ]  <-- u_i = e_i * s_i^-1 mod n

print(f"\nConstructing Lattice with {len(signatures)} signatures...")
dim = len(signatures) + 2
M = Matrix(ZZ, dim, dim)
scale = 2**(bits_known + 1) # scaling factor to balance the lattice

for i in range(len(signatures)):
    r, s, e = signatures[i]
    s_inv = inverse_mod(s, order)
    t = (r * s_inv) % order
    u = (e * s_inv - bound//2) % order  # recenter: u = e/s - bound/2 mod n
    
    # HNP Lattice construction
    M[i, i] = order * scale
    M[dim-2, i] = t * scale
    M[dim-1, i] = u * scale

# weights to balance the lattice
M[dim-2, dim-2] = 1 
M[dim-1, dim-1] = order

print("Running BKZ reduction...")
M_reduced = M.BKZ(block_size=20)

# extract private key
recovered_d = 0
for row in M_reduced:
    # row corresponding to the key
    if abs(row[dim-1]) == order:
        potential_d = abs(row[dim-2])
        if potential_d * G == Q:
            recovered_d = potential_d
            break
        elif (order - potential_d) * G == Q:
            recovered_d = order - potential_d
            break

print(f"Recovered Key: {recovered_d}")
print(f"Match ? {recovered_d == d_target}")

Target Private Key: 20180230641674394251968174366232623527370296021611257614398335595734926567456

Constructing Lattice with 42 signatures...
Running BKZ reduction...
Recovered Key: 20180230641674394251968174366232623527370296021611257614398335595734926567456
Match ? True


### Bonus: Differential Fault Analysis on ElGamal
Key recovery via differential analysis following a simulated hardware error.

In [26]:
# secp256k1
p = 2^256 - 2^32 - 2^9 - 2^8 - 2^7 - 2^6 - 2^4 - 1
E = EllipticCurve(GF(p), [0, 7])
Gx = 0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798
Gy = 0x483ADA7726A3C4655DA4FBFC0E1108A8FD17B448A68554199C47D08FFB10D4B8
order = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141
G = E(Gx, Gy)

# ElGamal Setup
priv_key = randint(1, order - 1)
pub_key = priv_key * G
print(f"Target Private Key: {priv_key}")

# encryption (Message M is a point on curve)
M_target = 1337 * G 
k = randint(1, order - 1)
C1 = k * G
C2 = k * pub_key + M_target

# decryption oracle with Fault Injection
def decrypt_oracle(C1, C2, fault=False):
    d_used = priv_key
    fault_pos = None
    
    if fault:
        # flip one random bit of the private key
        fault_pos = randint(0, 255)
        d_used = int(d_used) ^^ (1 << int(fault_pos))
        
    # decryption: M = C2 - d * C1
    S = d_used * C1
    M_dec = C2 - S
    return M_dec, fault_pos

print("Starting DFA attack...\n")
trials = 2000
recovered_bits = {}

# get reference decryption (no fault)
M_ref, _ = decrypt_oracle(C1, C2, fault=False)

for i in range(trials):
    if i % 100 == 0: print(f"Progress: {i}/{trials} trials...", end="\r")

    M_fault, pos = decrypt_oracle(C1, C2, fault=True)
    if pos is None: continue
    
    # difference: D = M_fault - M_ref
    # theory: M_ref = C2 - d*C1
    #         M_fault = C2 - d'*C1
    #         D = (d - d') * C1
    D = M_fault - M_ref
    
    # hypothesis check:
    #   if bit was 1: d' = d - 2^pos => d - d' = 2^pos
    #   if bit was 0: d' = d + 2^pos => d - d' = -2^pos
    
    scalar = 1 << pos
    if (scalar * C1) == D:
        recovered_bits[pos] = 1
    elif ((-scalar) % order) * C1 == D:
        recovered_bits[pos] = 0

reconstructed_key = 0
for pos, bit in recovered_bits.items():
    if bit:
        reconstructed_key |= (1 << pos)

missing_bits = [i for i in range(256) if i not in recovered_bits]
nb_missing = len(missing_bits)
print(f"Recovered {256 - nb_missing}/256 bits.")

if nb_missing == 0:
    print(f"Private key recovered: {reconstructed_key}")
    print(f"Match ? {reconstructed_key == priv_key}")
elif nb_missing <= 15:
    print(f"Brute-forcing {nb_missing} remaining bits...")
    for i in range(1 << nb_missing):
        cand = reconstructed_key
        for j, pos in enumerate(missing_bits):
            if (i >> j) & 1:
                cand |= (1 << pos)
        if cand == priv_key:
            print(f"Private key recovered: {cand}")
            print(f"Match ? {cand == priv_key}")
            break
else:
    print(f"Too many missing bits ({nb_missing}) to brute-force.")

Target Private Key: 110437442817680201492137322617123603614378302167534458976194375864141891404291
Starting DFA attack...

Recovered 256/256 bits.als...
Private key recovered: 110437442817680201492137322617123603614378302167534458976194375864141891404291
Match ? True
